In [2]:
from manim import *
import numpy as np

config.media_width = "75%"
config.verbosity = "WARNING"

/usr/local/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
%%manim -ql PlotNormal

class PlotNormal(ThreeDScene):
    def construct(self):
        resolucao = 24
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Função da normal bivariada (f : R^2 \to R)
        def normal(u, v):
            w = np.exp(-(u ** 2 + v ** 2))
            return np.array([u, v, w])

        # Cria a superfície com a resolução de interpolação dada
        normal_sup = Surface(
            normal,
            resolution=(resolucao, resolucao),
            v_range=[-2, +2],
            u_range=[-2, +2]
        )

        # Roda a câmera
        self.begin_ambient_camera_rotation(rate=0.4)

        normal_sup.scale(2, about_point=ORIGIN)
        normal_sup.set_style(fill_opacity=1, stroke_color=GREEN)
        eixos = ThreeDAxes()
        self.add(eixos, normal_sup)
        self.wait(2)

Manim Community v0.19.0

In [9]:
%%manim -ql OtimizacaoRestricao

class OtimizacaoRestricao(ThreeDScene):
    def construct(self):
        # Configuração inicial da câmera
        self.set_camera_orientation(phi=75 * DEGREES, theta=-60 * DEGREES)

        # Eixos 3D
        eixos = ThreeDAxes(
            x_range=[-2.5, 2.5], y_range=[-3.5, 3.5], z_range=[0, 15],
            x_length=6, y_length=6, z_length=3
        )
        self.play(Create(eixos), run_time=2)

        # Função f(x, y) = x^2 + y^2
        def f(x, y):
            return x**2 + y**2

        # Superfície f(x,y) sobre o domínio elíptico
        superficie = Surface(
            lambda u, v: eixos.coords_to_point(
                2 * np.cos(u) * np.cos(v),
                3 * np.sin(u) * np.cos(v),
                f(2 * np.cos(u) * np.cos(v), 3 * np.sin(u) * np.cos(v))
            ),
            u_range=[0, TAU],
            v_range=[0, 0.5],
            resolution=(30, 15),
            fill_opacity=0.6,
            checkerboard_colors=[BLUE_E, BLUE_D],
        )
        self.play(Create(superficie), run_time=3)

        # Domínio elíptico (projeção no plano xy)
        elipse = ParametricFunction(
            lambda t: eixos.coords_to_point(2 * np.cos(t), 3 * np.sin(t), 0),
            t_range=[0, TAU],
            color=GREEN,
            stroke_width=4
        )
        self.play(Create(elipse), run_time=2)
        self.wait(1)

        # Parametrização do bordo da elipse
        def gamma(t):
            return np.array([2 * np.cos(t), 3 * np.sin(t)])

        # Imagem da borda no gráfico
        borda_imagem = ParametricFunction(
            lambda t: eixos.coords_to_point(*gamma(t), f(*gamma(t))),
            t_range=[0, TAU],
            color=RED,
            stroke_width=4
        )
        self.play(Create(borda_imagem), run_time=3)
        self.wait(2)

        # Apagar elementos 3D
        self.play(
            FadeOut(superficie),
            FadeOut(elipse),
            FadeOut(borda_imagem),
            FadeOut(eixos),
            run_time=2
        )

        # Reconfigurar a câmera para visualização do gráfico 2D
        self.set_camera_orientation(phi=0 * DEGREES, theta=-90* DEGREES)

        # Eixos 2D
        eixos2d = Axes(
            x_range=[0, TAU],
            y_range=[0, 15],
            x_length=6,
            y_length=4,
            axis_config={"include_tip": True},
        ).to_edge(DOWN)

        xlabel = eixos2d.get_x_axis_label(MathTex("t"), edge=RIGHT, direction=RIGHT)
        ylabel = eixos2d.get_y_axis_label(MathTex(r"(f \circ \alpha)(t)"), edge=UP, direction=UP)

        self.play(Create(eixos2d), FadeIn(xlabel), FadeIn(ylabel), run_time=2)

        titulo = Title(r"Gráfico da função composta $(f \circ \alpha)(t)$")
        self.play(Write(titulo), run_time=2)
        self.wait(1)

        # Função composta f(γ(t)) = 4cos²(t) + 9sin²(t)
        def f_composicao(t):
            x, y = gamma(t)
            return f(x, y)

        curva_composta = ParametricFunction(
            lambda t: eixos2d.coords_to_point(t, f_composicao(t)),
            t_range=[0, TAU],
            color=YELLOW,
            stroke_width=3
        )
        self.play(Create(curva_composta), run_time=3)
        self.wait(4)

Manim Community v0.19.0

In [4]:
%%manim -ql Lorenz

class Lorenz(ThreeDScene):
    def construct(self):
        self.camera.background_color = DARKER_GRAY
        
        cond_iniciais = [np.array([-1.0, 1.0, 0.0]), 
                         np.array([-0.7, 1.0, 0.0]), 
                         np.array([-1.0, 0.7, 0.0])]
        escala = 0.1

        # A quantidade de segundos que se passa entre cada integração
        epsilon = 0.01

        # Cores de cada PVI
        cores = [RED_B, ORANGE, TEAL]
        curvas = VGroup()

        # Para cada combinação de valor inicial e cor, crie uma curva
        for pos_ini, cor in zip(cond_iniciais, cores):
            f = self.trajetoria(pos_ini, dt=epsilon)
            curva = ParametricFunction(
                lambda t: f(t),
                color=cor,
                t_range=np.array([0, 30, epsilon]),  # Tempo de simulação é de 0 a 30 segundos, com cálculos a cada epsilon segundos e interpolados entre si
                use_vectorized=False
            )
            curva.scale(escala)

            # Centraliza todas as curvas
            curva.move_to(ORIGIN)
            curvas.add(curva)

        self.set_camera_orientation(phi=80 * DEGREES, theta=45 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.1)

        # Equações do sistema de Lorenz. O manim irá, por baixo do tapete, compilar separadamente
        # as letras x, y, z para colori-las separadamente (e por isso o uso do \over ao invés de \frac)
        equacoes = MathTex(
            r"""
            {\mathrm{d} x} \over {\mathrm{d} t} &= \sigma(y - x) \\
            {\mathrm{d} y} \over {\mathrm{d} t} &= x(\rho - z) - y \\
            {\mathrm{d} z} \over {\mathrm{d} t} &= xy - \beta z
            """,
            substrings_to_isolate=["x", "y", "z"]
        ).scale(0.6)
        equacoes.set_color_by_tex('x', RED)
        equacoes.set_color_by_tex('y', GREEN)
        equacoes.set_color_by_tex('z', BLUE)

        # Mostre as equações no canto superior esquerdo
        equacoes.to_edge(UL)

        # Defina que as equações não devem se mover junto com a câmera
        self.add_fixed_in_frame_mobjects(equacoes)
        self.play(Write(equacoes), run_time=1.5)

        # Crie todas as curvas na lista simultaneamente
        self.play(*[Create(c) for c in curvas], run_time=20, rate_func=linear)

    # Retorna as equações diferenciais do atrator de Lorenz.
    def lorenz_system(self, pos, sigma=10, rho=28, beta=8 / 3):
        x, y, z = pos
        dx_dt = sigma * (y - x)
        dy_dt = x * (rho - z) - y
        dz_dt = x * y - beta * z
        return np.array([dx_dt, dy_dt, dz_dt])

    # Um passo do método de Runge-Kutta de 4ª ordem
    def rk4_step(self, pos, dt):
        k1 = self.lorenz_system(pos)
        k2 = self.lorenz_system(pos + dt * k1 / 2)
        k3 = self.lorenz_system(pos + dt * k2 / 2)
        k4 = self.lorenz_system(pos + dt * k3)
        return pos + dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6

    # Retorna uma função f(t) que dá a posição no tempo t
    # a partir de integração numérica, evitando reintegrar se esse cálculo já foi feito antes
    def trajetoria(self, start_pos, dt=0.01):
        cache = {0: np.array(start_pos)}
        def f(t):
            # arredonda t para múltiplos de dt
            steps = int(np.round(t / dt))
            if steps in cache:
                return cache[steps]
            # integra até o passo desejado
            pos = cache[max(cache.keys())]
            for i in range(max(cache.keys()) + 1, steps + 1):
                pos = self.rk4_step(pos, dt)
                cache[i] = pos
            return cache[steps]
        return f

Manim Community v0.19.0